In [47]:
import pandas as pd
import numpy as np
import seawater as sw
import PyCO2SYS as pyco2
import re

from calendar import month_name

In [48]:
pd.options.display.max_columns = None

In [49]:
def NN_fill(df, seriesname, thresholds={"longitude": 0.05, "latitude": 0.05, "depth": 5}):
    df.sort_values(by=["longitude", "latitude", "datetime", "depth"], inplace=True)
    N, _ = df.shape
    X = df[seriesname].copy()
    def fill(upward=True):
        if upward:
            X_shifted = np.concatenate([[np.NaN], X[:-1]])
            shift = 1
        else:
            X_shifted = np.concatenate([X[1:], [np.NaN]])
            shift = -1
        mask = np.isnan(X) & (~np.isnan(X_shifted))
        for name, threshold in thresholds.items():
            mask &= (df[name].diff(shift).abs() < threshold)
        return mask, df["depth"].diff(shift).abs(), X_shifted

    upmask, updiffs, Xup = fill(upward=True)
    downmask, downdiffs, Xdown = fill(upward=False)
    X[upmask & (~downmask)] = Xup[upmask & (~downmask)]
    X[(~upmask) & downmask] = Xdown[(~upmask) & downmask]
    X[upmask & downmask & (updiffs <= downdiffs)] = Xup[upmask & downmask & (updiffs <= downdiffs)]
    X[upmask & downmask & (updiffs > downdiffs)] = Xdown[upmask & downmask & (updiffs > downdiffs)]

    return X

In [50]:
def read_emodnet(filepath,
                 flag,
                 defaults={"phosphate": 0.2, "silicate": 5.0},
                 nn_thresholds={"longitude": 0.01, "latitude": 0.01, "depth": 5},
                 input_columns=[
                    "datetime", "longitude", "latitude", "depth",
                    "phosphate", "silicate", "nitrate", "oxygen",
                    "alkalinity", "dic",
                    "temperature", "salinity",
                    "instrument_type", "cruise_id"],
                 output_columns=[
                     "YYYY", "MM", "DD", "lat", "lon", "depth",
                     "phosphate", "silicate", "nitrate", "oxygen", 
                     "ALK", "DIC",
                     "temp", "pressure", "density", "salinity",
                     "PHt_{T-Press-ins}_ric", "pCO2", "xCO2",
                     "ID_type_profile", "cruise_id", "flag"],
                 columns_mapping={
                     "latitude": "lat",
                     "longitude": "lon",
                     "dic": "DIC",
                     "alkalinity": "ALK",
                     "temperature": "temp",
                     "instrument_type": "ID_type_profile"}):

    df = pd.read_parquet(filepath, columns=input_columns)
    
    df["nn_temperature"] = NN_fill(df, "temperature", thresholds=nn_thresholds)
    df["nn_salinity"] = NN_fill(df, "salinity", thresholds=nn_thresholds)
    df["pressure"] = sw.pres(df.depth, df.latitude)
    df["density"] = sw.dens(df.nn_salinity, df.nn_temperature, df.pressure)
    df["alkalinity"] = 1e6 * df.alkalinity / df.density
    df["dic"] = 1e3 * df.dic / df.density
    df["phosphate"] = 1e3 * df.phosphate / df.density
    df["silicate"] = 1e3 * df.silicate / df.density
    df["nitrate"] = 1e3 * df.nitrate / df.density
    df["oxygen"] = 1e3 * df.oxygen / df.density

    for key, val in defaults.items():
        mask = ~np.isnan(df[key])
        new_key = key + "_with_defaults"
        df.loc[mask, new_key] = df.loc[mask, key]
        df.loc[~mask, new_key] = np.float32(val)

    ric = pyco2.sys(
        par1=df["alkalinity"],
        par2=df["dic"],
        par1_type=1,
        par2_type=2,
        salinity=df["nn_salinity"],
        temperature=df["nn_temperature"],
        pressure=df["pressure"],
        total_silicate=df["silicate_with_defaults"],
        total_phosphate=df["phosphate_with_defaults"],
        opt_pH_scale=1,
        opt_k_carbonic=4,
        opt_k_bisulfate=1,
        opt_total_borate=1,
        opt_k_fluoride=1)

    df["PHt_{T-Press-ins}_ric"] = ric['pH_total']
    df["pCO2"] = ric['pCO2']
    df["xCO2"] = ric['xCO2']

    df["YYYY"] = df.datetime.map(lambda dt: dt.year)
    df["MM"] = df.datetime.map(lambda dt: dt.month)
    df["DD"] = df.datetime.map(lambda dt: dt.day)
    df["instrument_type"] = df.instrument_type.map(lambda b: b.decode("utf-8"))
    df["cruise_id"] = df.cruise_id.map(lambda b: b.decode("utf-8"))
    df["flag"] = flag

    df.rename(columns=columns_mapping, inplace=True)

    return df[output_columns]

In [51]:
emodnet = read_emodnet("../1999-2023/parquet/emodnet_profile.parquet", "1-emodnet")

In [52]:
def parse_instrument_type(t):
    if re.match("1", t) is not None:
        return "nut"
    else:
        return "probe"

def parse_month(m):
    if re.fullmatch(r'[0-9]+\.0+', m):
        return float(m)
    else:
        search = map(lambda s: re.match(f"(?i){m}", s) is not None, month_name)
        return float(list(search).index(True))

def read_cruises(filename,
                 flag,
                 idcampains,
                 output_columns=[
                    "YYYY", "MM", "DD", "lat", "lon", "depth",
                    "phosphate", "silicate", "nitrate", "oxygen", 
                    "ALK", "DIC", "DICric",
                    "temp", "pressure", "density", "salinity", 
                    "PHt_{T-Press-ins}_obs", "PHt_{T-Press-ins}_ric", "pH25_T", "pCO2", "xCO2",
                    "cruise_id", "flag"],
                columns_mapping={
                     "year": "YYYY",
                     "month": "MM",
                     "day": "DD"}):
    df = pd.read_csv(filename, converters={"month": parse_month, "ID_type_profile": parse_instrument_type})
    df.rename(columns=columns_mapping, inplace=True)
    df["pressure"] = sw.pres(df.depth, df.lat)
    df["YYYY"] = df.YYYY.map(int)
    df["MM"] = df.MM.map(int)
    df["DD"] = df.DD.map(int)
    df["idcampain"] = df.dataset.map(int)
    df["cruise_id"] = np.fromiter(map(lambda n: idcampains[n], df.idcampain - 1), dtype=object)
    df["flag"] = flag
    return df[output_columns]

In [53]:
vdb_idcampains = ['TALPRO', 'SOMBA', 'PEACETIME', 'MEDWAVES', 'MSM72', 'GIANI']
vdb_cruises = read_cruises("../1999-2023/csv/df_Carbon_cruises_Med_New_newConfig.csv", "2-cruises_new", vdb_idcampains)

gpc_idcampains = ["METEOR51","BOUM 2008","METEOR95","PROSOPE","METEOR","EGEO APRIL","EGEO SEPT","REGINA MARIS", 
                  "Garcia del Cid","SESAME - ADRIATICO 2008","CARBOGIB 1","CARBOGIB 2","CARBOGIB 3","CARBOGIB 4", 
                  "CARBOGIB 5","CARBOGIB 6","GIFT 1","GIFT 2","DYFAMED","MEDSEA 2013"]
gpc_cruises = read_cruises("../1999-2023/csv/df_Carbon_cruises_Med_GP_newConfig.csv", "3-cruises_old", gpc_idcampains)

cruises = pd.concat([vdb_cruises, gpc_cruises], ignore_index=True)
cruises

,YYYY,MM,DD,lat,lon,depth,phosphate,silicate,nitrate,oxygen,ALK,DIC,DICric,temp,pressure,density,salinity,PHt_{T-Press-ins}_obs,PHt_{T-Press-ins}_ric,pH25_T,pCO2,xCO2,cruise_id,flag
0,2016,8,19,38.3000,13.3900,462.475261,0.26,4.49,4.65,NaN,NaN,NaN,NaN,14.350,466.660261,1031.120264,38.820,NaN,NaN,NaN,NaN,NaN,TALPRO,2-cruises_new
1,2016,8,19,38.3000,13.3900,398.791751,0.25,4.14,4.96,NaN,NaN,NaN,NaN,14.393,402.342764,1030.831613,38.824,NaN,NaN,NaN,NaN,NaN,TALPRO,2-cruises_new
2,2016,8,19,38.3000,13.3900,299.612310,0.21,3.18,4.53,NaN,NaN,NaN,NaN,14.629,302.212715,1030.360032,38.852,NaN,NaN,NaN,NaN,NaN,TALPRO,2-cruises_new
3,2016,8,19,38.3000,13.3900,198.996911,0.19,2.58,4.15,NaN,NaN,NaN,NaN,14.738,200.678630,1029.858874,38.813,NaN,NaN,NaN,NaN,NaN,TALPRO,2-cruises_new
4,2016,8,19,38.3000,13.3900,100.316211,0.10,0.84,1.18,NaN,NaN,NaN,NaN,14.877,101.141545,1028.967504,38.267,NaN,NaN,NaN,NaN,NaN,TALPRO,2-cruises_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8002,2013,5,31,40.9502,3.3204,65.000000,NaN,NaN,NaN,NaN,2585.0,2265.0,2265.0,14.500,65.545338,NaN,38.340,NaN,8.152536,NaN,326.498273,331.788007,MEDSEA 2013,3-cruises_old
8003,2013,5,31,40.9502,3.3204,50.000000,NaN,NaN,NaN,NaN,2590.0,2258.0,2258.0,15.160,50.417791,NaN,38.300,NaN,8.161276,NaN,319.576511,324.983182,MEDSEA 2013,3-cruises_old
8004,2013,5,31,40.9502,3.3204,25.000000,NaN,NaN,NaN,NaN,2558.0,2241.0,2241.0,16.610,25.207479,NaN,38.290,NaN,8.120780,NaN,353.814183,360.392687,MEDSEA 2013,3-cruises_old
8005,2013,5,31,40.9502,3.3204,10.000000,NaN,NaN,NaN,NaN,2556.0,2236.0,2236.0,16.610,10.082652,NaN,38.260,NaN,8.126690,NaN,348.378617,354.856165,MEDSEA 2013,3-cruises_old


In [54]:
data = pd.concat([emodnet, cruises])
data.sort_values(by=["YYYY", "MM", "DD", "lat", "lon", "depth", "flag"], inplace=True)
data.drop_duplicates(subset=["YYYY", "MM", "DD", "lat", "lon", "depth"], inplace=True, ignore_index=True)
data

,YYYY,MM,DD,lat,lon,depth,phosphate,silicate,nitrate,oxygen,ALK,DIC,temp,pressure,density,salinity,PHt_{T-Press-ins}_ric,pCO2,xCO2,ID_type_profile,cruise_id,flag,DICric,PHt_{T-Press-ins}_obs,pH25_T
0,1998,2,5,43.416700,7.866700,5.0,NaN,NaN,NaN,NaN,2536.2,2243.9,NaN,5.042412,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2243.9,NaN,NaN
1,1998,2,5,43.416700,7.866700,10.0,NaN,NaN,NaN,NaN,2540.5,2246.5,NaN,10.084938,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2246.5,NaN,NaN
2,1998,2,5,43.416700,7.866700,20.0,NaN,NaN,NaN,NaN,2543.2,2252.3,NaN,20.170330,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2252.3,NaN,NaN
3,1998,2,5,43.416700,7.866700,30.0,NaN,NaN,NaN,NaN,2539.5,2248.7,NaN,30.256175,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2248.7,NaN,NaN
4,1998,2,5,43.416700,7.866700,50.0,NaN,NaN,NaN,NaN,2543.2,2251.7,NaN,50.429226,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2251.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128036,2021,10,27,42.436333,18.591633,23.0,0.106149,1.740258,0.581384,218.209930,NaN,NaN,20.799999,23.194567,1026.859253,37.900002,NaN,NaN,NaN,probe,227,1-emodnet,NaN,NaN,NaN
128037,2022,1,19,42.431732,18.600517,8.0,0.051555,7.060152,1.542767,225.258698,NaN,NaN,13.900000,8.064157,1028.023193,37.299999,NaN,NaN,NaN,probe,228,1-emodnet,NaN,NaN,NaN
128038,2022,1,19,42.431732,18.600517,15.0,0.051526,8.897547,1.394130,265.237396,NaN,NaN,14.600000,15.130409,1028.598145,38.200001,NaN,NaN,NaN,probe,228,1-emodnet,NaN,NaN,NaN
128039,2022,1,19,42.436333,18.591633,12.0,0.060283,3.646178,1.094826,199.635712,NaN,NaN,14.400000,12.096236,1028.474243,38.000000,NaN,NaN,NaN,probe,228,1-emodnet,NaN,NaN,NaN


In [55]:
data.loc[~np.isnan(data.DIC), "DICric"] = data.loc[~np.isnan(data.DIC), "DIC"]
data

,YYYY,MM,DD,lat,lon,depth,phosphate,silicate,nitrate,oxygen,ALK,DIC,temp,pressure,density,salinity,PHt_{T-Press-ins}_ric,pCO2,xCO2,ID_type_profile,cruise_id,flag,DICric,PHt_{T-Press-ins}_obs,pH25_T
0,1998,2,5,43.416700,7.866700,5.0,NaN,NaN,NaN,NaN,2536.2,2243.9,NaN,5.042412,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2243.9,NaN,NaN
1,1998,2,5,43.416700,7.866700,10.0,NaN,NaN,NaN,NaN,2540.5,2246.5,NaN,10.084938,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2246.5,NaN,NaN
2,1998,2,5,43.416700,7.866700,20.0,NaN,NaN,NaN,NaN,2543.2,2252.3,NaN,20.170330,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2252.3,NaN,NaN
3,1998,2,5,43.416700,7.866700,30.0,NaN,NaN,NaN,NaN,2539.5,2248.7,NaN,30.256175,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2248.7,NaN,NaN
4,1998,2,5,43.416700,7.866700,50.0,NaN,NaN,NaN,NaN,2543.2,2251.7,NaN,50.429226,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2251.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128036,2021,10,27,42.436333,18.591633,23.0,0.106149,1.740258,0.581384,218.209930,NaN,NaN,20.799999,23.194567,1026.859253,37.900002,NaN,NaN,NaN,probe,227,1-emodnet,NaN,NaN,NaN
128037,2022,1,19,42.431732,18.600517,8.0,0.051555,7.060152,1.542767,225.258698,NaN,NaN,13.900000,8.064157,1028.023193,37.299999,NaN,NaN,NaN,probe,228,1-emodnet,NaN,NaN,NaN
128038,2022,1,19,42.431732,18.600517,15.0,0.051526,8.897547,1.394130,265.237396,NaN,NaN,14.600000,15.130409,1028.598145,38.200001,NaN,NaN,NaN,probe,228,1-emodnet,NaN,NaN,NaN
128039,2022,1,19,42.436333,18.591633,12.0,0.060283,3.646178,1.094826,199.635712,NaN,NaN,14.400000,12.096236,1028.474243,38.000000,NaN,NaN,NaN,probe,228,1-emodnet,NaN,NaN,NaN


In [56]:
def unique(lst):
    return list(set(lst))

valid_ranges = {"ALK": (1e3, 4e3), "DIC": (1e3, 4e3), "PHt_{T-Press-ins}_ric": (7, 10)}

for (name, (a, b)) in valid_ranges.items():
    indices = (data[name] < a) & (data[name] > b)
    data.loc[indices, unique([name, "PHt_{T-Press-ins}_ric"])] = np.NaN

valid_indices = np.full(len(data), False)
for name in ["ALK", "DIC"]:
    valid_indices = valid_indices | (~np.isnan(data[name]))

data = data.loc[valid_indices].copy()
data.reset_index(inplace=True)

data["idcampain"] = pd.Categorical(data.cruise_id).codes

In [57]:
data

,index,YYYY,MM,DD,lat,lon,depth,phosphate,silicate,nitrate,oxygen,ALK,DIC,temp,pressure,density,salinity,PHt_{T-Press-ins}_ric,pCO2,xCO2,ID_type_profile,cruise_id,flag,DICric,PHt_{T-Press-ins}_obs,pH25_T,idcampain
0,0,1998,2,5,43.416700,7.866700,5.000000,NaN,NaN,NaN,NaN,2536.200000,2243.9,NaN,5.042412,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2243.9,NaN,NaN,7
1,1,1998,2,5,43.416700,7.866700,10.000000,NaN,NaN,NaN,NaN,2540.500000,2246.5,NaN,10.084938,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2246.5,NaN,NaN,7
2,2,1998,2,5,43.416700,7.866700,20.000000,NaN,NaN,NaN,NaN,2543.200000,2252.3,NaN,20.170330,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2252.3,NaN,NaN,7
3,3,1998,2,5,43.416700,7.866700,30.000000,NaN,NaN,NaN,NaN,2539.500000,2248.7,NaN,30.256175,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2248.7,NaN,NaN,7
4,4,1998,2,5,43.416700,7.866700,50.000000,NaN,NaN,NaN,NaN,2543.200000,2251.7,NaN,50.429226,NaN,NaN,NaN,NaN,NaN,NaN,DYFAMED,3-cruises_old,2251.7,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,128007,2021,7,1,42.664165,5.198167,59.517197,0.163237,3.481408,5.635548,202.702133,2571.111572,NaN,13.709,60.022686,1029.181274,38.445000,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46
7315,128011,2021,7,2,42.862999,5.199667,66.061821,0.021381,1.704626,1.530665,224.979813,2542.366455,NaN,13.973,66.630440,1028.964600,38.201000,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46
7316,128012,2021,7,2,42.862999,5.199667,79.648468,0.042756,2.192221,3.582765,208.127823,2566.849365,NaN,13.873,80.331413,1029.093384,38.261002,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46
7317,128013,2021,7,2,42.862999,5.199667,98.390610,0.118533,2.560127,4.842381,202.465378,2602.121826,NaN,13.839,99.237686,1029.245605,38.340000,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46


In [58]:
data.to_parquet("../1999-2023_carbonatic.parquet")

In [59]:
pd.read_parquet("../1999-2023_carbonatic.parquet")

,index,YYYY,MM,DD,lat,lon,depth,phosphate,silicate,nitrate,oxygen,ALK,DIC,temp,pressure,density,salinity,PHt_{T-Press-ins}_ric,pCO2,xCO2,ID_type_profile,cruise_id,flag,DICric,PHt_{T-Press-ins}_obs,pH25_T,idcampain
0,0,1998,2,5,43.416700,7.866700,5.000000,NaN,NaN,NaN,NaN,2536.200000,2243.9,NaN,5.042412,NaN,NaN,NaN,NaN,NaN,None,DYFAMED,3-cruises_old,2243.9,NaN,NaN,7
1,1,1998,2,5,43.416700,7.866700,10.000000,NaN,NaN,NaN,NaN,2540.500000,2246.5,NaN,10.084938,NaN,NaN,NaN,NaN,NaN,None,DYFAMED,3-cruises_old,2246.5,NaN,NaN,7
2,2,1998,2,5,43.416700,7.866700,20.000000,NaN,NaN,NaN,NaN,2543.200000,2252.3,NaN,20.170330,NaN,NaN,NaN,NaN,NaN,None,DYFAMED,3-cruises_old,2252.3,NaN,NaN,7
3,3,1998,2,5,43.416700,7.866700,30.000000,NaN,NaN,NaN,NaN,2539.500000,2248.7,NaN,30.256175,NaN,NaN,NaN,NaN,NaN,None,DYFAMED,3-cruises_old,2248.7,NaN,NaN,7
4,4,1998,2,5,43.416700,7.866700,50.000000,NaN,NaN,NaN,NaN,2543.200000,2251.7,NaN,50.429226,NaN,NaN,NaN,NaN,NaN,None,DYFAMED,3-cruises_old,2251.7,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7314,128007,2021,7,1,42.664165,5.198167,59.517197,0.163237,3.481408,5.635548,202.702133,2571.111572,NaN,13.709,60.022686,1029.181274,38.445000,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46
7315,128011,2021,7,2,42.862999,5.199667,66.061821,0.021381,1.704626,1.530665,224.979813,2542.366455,NaN,13.973,66.630440,1028.964600,38.201000,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46
7316,128012,2021,7,2,42.862999,5.199667,79.648468,0.042756,2.192221,3.582765,208.127823,2566.849365,NaN,13.873,80.331413,1029.093384,38.261002,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46
7317,128013,2021,7,2,42.862999,5.199667,98.390610,0.118533,2.560127,4.842381,202.465378,2602.121826,NaN,13.839,99.237686,1029.245605,38.340000,NaN,NaN,NaN,probe,MOOSE-GE 2021,1-emodnet,NaN,NaN,NaN,46
